# Proyecto 1: Brecha Digital
### Análisis de datos

#### Basado en el reporte ["La Brecha Digital en California"](https://www.ppic.org/wp-content/uploads/jtf-californias-digital-divide-in-spanish.pdf)

## Pregunta(s) de Investigación:
1. ¿Qué porcentaje de hogares en el estado X tiene acceso a internet de alta velocidad?
2. ¿Varía este número según los grupos demográficos? (en este caso raza/etnia).

## Meta:
* Usar nuestro conjunto de datos `datos_analisis` (creado en el [notebook Preparación de Datos](00_BrechaDigital_PrepDatos.ipynb)) para contestar nuestras preguntas de investación.

## Contexto:
* Escribe tu propia descripción del contexto: En que parte del proyecto encaja este notebook, una pequeña descripción de los datos que vas a utilizar (_este conjunto de datos contiene información de X estado para el año YYYY)_.

***

#### Paso 1: Prepara tu entorno de trabajo.

**Import**a las bibliotecas necesarias y crea objetos `Path` (ruta de archivo). Esto grarantiza reproducibilidad en distintos sistemas operativos (Windows utiliza `\` en lugar de `/` para separar los nombres de archivos.

Necesitamos: 
1. `pandas` para trabajar con los datos.
2. `pathlib`, y más específicamente su objeto `Path`, para trabajar con rutas de archivos. Esto asegura que nuestro código funcione en Windows (que utiliza `\` en sus rutas) y MacOS/Linux (los cuales utilizan `/`).
3. `datetime` - tip: Existen sistemas de control de versiones para datos pero etiquetar tus archivos de datos (cuando no son masivos) no es un mal primer paso si estás comenzando.
4. `arbol` - para mostrar el árbol de directorios.

In [1]:
# Preparando tu entorno de trabajo
import pandas as pd
from pathlib import Path
from herramientas import arbol
from datetime import datetime as dt
hoy = dt.today().strftime("%d-%m-%y")

print(hoy)

06-05-19


In [2]:
# Directorio de datos y rutas
RUTA_DATOS_EN_BRUTO = Path("../datos/brutos/")
RUTA_DATOS_INTERINOS = Path("../datos/interinos/")
RUTA_DATOS_PROCESADOS = Path("../datos/procesados/")
RUTA_DATOS_FINALES = Path("../datos/finales/")

In [3]:
arbol(RUTA_DATOS_INTERINOS)

+ ..\datos\interinos
    + datos_estatales-06-05-19.dta
    + datos_para_analisis-06-05-19.dta
    + placeholder
    + state_data-01-May-19.dta
    + working_data-01-May-19.dta


In [4]:
datos = pd.read_stata(RUTA_DATOS_INTERINOS / f'datos_para_analisis-{hoy}.dta')

In [5]:
datos.shape

(44816, 18)

In [6]:
datos.head()

,year,serial,hhwt,statefip,countyfip,gq,cinethh,cihispeed,pernum,perwt,relate,related,sex,age,race,raced,hispan,hispand
0,2017,953662,57,ohio,0,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",1,58,head/householder,head/householder,female,48,white,white,not hispanic,not hispanic
1,2017,953662,57,ohio,0,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",2,62,child,child,male,20,white,white,not hispanic,not hispanic
2,2017,953662,57,ohio,0,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",3,78,child,child,female,9,white,white,not hispanic,not hispanic
3,2017,953668,140,ohio,61,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",1,140,head/householder,head/householder,male,28,black/african american/negro,black/african american/negro,not hispanic,not hispanic
4,2017,953668,140,ohio,61,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",2,192,sibling,sibling,female,16,black/african american/negro,black/african american/negro,not hispanic,not hispanic


In [7]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44816 entries, 0 to 44815
Data columns (total 18 columns):
year         44816 non-null category
serial       44816 non-null int32
hhwt         44816 non-null int16
statefip     44816 non-null category
countyfip    44816 non-null int16
gq           44816 non-null category
cinethh      44816 non-null category
cihispeed    44816 non-null category
pernum       44816 non-null int8
perwt        44816 non-null int16
relate       44816 non-null category
related      44816 non-null category
sex          44816 non-null category
age          44816 non-null category
race         44816 non-null category
raced        44816 non-null category
hispan       44816 non-null category
hispand      44816 non-null category
dtypes: category(13), int16(3), int32(1), int8(1)
memory usage: 1.4 MB


Nuestra **unidad de observación** es todavía una persona (ponderada) pero nos interesa datos a nivel **hogar**.

De la [documentación](https://usa.ipums.org/usa-action/variables/HHWT#description_section) de IPUMS:
>HHWT indica cuantos hogares de la población de EEUU estan siendo representados por cada hogar en una muestra de IPUMS.<br><br>
>Es generalmente una buena idea usar HHWT cuando se está llevando a cabo análisis a nivel hogar de cualquier muestra de IPUMS. El uso de HHWT es opcional cuando se están analizando uno de las muestras "planas" o no ponderados de IPUMS. Muestras planas de IPUMS incluyen las muestras 1% de 1850-1930, todas las muestras de 1960, 1970 y 1980, las muestras no ponderadas 1% de 1990 y 2000, la muestra 10% de 2010, y cualquiera de los conjuntos de datos del censo de cuentas totales 100%. HHWT debe ser utilizada para obtener estadísticas representativas a nivel nacional para análisis a nivel hogar de cualquier muestra fuera de esas. <br><br>
>**Usuarios deberían también asegurarse de seleccionar una persona (por ejemplo, PERNUM = 1) para representar el hogar entero.**

***

#### Paso 2: Elimina todas las observaciones donde `pernum` no sea igual a 1

In [8]:
mascara_pernum = (datos['pernum'] == 1)

In [9]:
datos[mascara_pernum].shape

(11109, 18)

Guarda tus datos en una variable con un nombre apropiado.

In [10]:
hogares_del_estado = datos[mascara_pernum]

***

#### Paso 3: Familiarizate con tus variables de interés

De la [documentación](https://usa.ipums.org/usa-action/variables/CINETHH#description_section) de IPUMS:
>CINETHH reporta si algún miembro del hogar accede al internet. Aquí, "acceso" se refiere a si alguien en el hogar usa o se conecta al internet o no, sin importar si paga por el servicio o no.

In [11]:
# encuentra los totales de los valores de CINETHH
hogares_del_estado['cinethh'].value_counts()

yes, with a subscription to an internet service                10442
no internet access at this house, apartment, or mobile home      476
yes, without a subscription to an internet service               191
Name: cinethh, dtype: int64

De la [documentación](https://usa.ipums.org/usa-action/variables/CIHISPEED#description_section) de IPUMS:
>CIHISPEED reporta si la persona respondiendo o cualquier miembro de su hogar se subscribió al internet utilizando servicios de banda ancha (alta velocidad) tales como cable, fibra optica, o servicio DSL. <br><br>
>Nota al usuario: La ACS de 2016 introdujo cambios a las preguntas que pertienen al uso de computadoras y acceso al internet. Mira la sección comparabilidad y el texto del cuestionario para más información. Información adicional proveniente del Buró del Censo sobre la alteración de estas preguntas esta disponible en su reporte: **ACS Content Test Shows Need to Update Terminology**.

In [12]:
# encuentra los totales de los valores de CIHISPEED
hogares_del_estado['cihispeed'].value_counts()

yes (cable modem, fiber optic or dsl service)    8920
no                                               1522
n/a (gq)                                          667
Name: cihispeed, dtype: int64

<details>
    <summary>¿Adivina que puede hacer <span style="font-family:monospace">.value_counts()</span></summary>
    Tiene la opción de activar su parametro <span style="font-family:monospace">normalize</span> que normaliza tus series.
</details>

In [13]:
pd.Series.value_counts?

Signature:
pd.Series.value_counts(
    self,
    normalize=False,
    sort=True,
    ascending=False,
    bins=None,
    dropna=True,
)
Docstring:
Return a Series containing counts of unique values.

The resulting object will be in descending order so that the
first element is the most frequently-occurring element.
Excludes NA values by default.

Parameters
----------
normalize : boolean, default False
    If True then the object returned will contain the relative
    frequencies of the unique values.
sort : boolean, default True
    Sort by values.
ascending : boolean, default False
    Sort in ascending order.
bins : integer, optional
    Rather than count values, group them into half-open bins,
    a convenience for ``pd.cut``, only works with numeric data.
dropna : boolean, default True
    Don't include counts of NaN.

Returns
-------
counts : Series

See Also
--------
Series.count: Number of non-NA elements in a Series.
DataFrame.count: Number of non-NA elements in a DataFrame.



In [14]:
# pruebalo en la serie cinethh
hogares_del_estado['cinethh'].value_counts(normalize=True)

yes, with a subscription to an internet service                0.939959
no internet access at this house, apartment, or mobile home    0.042848
yes, without a subscription to an internet service             0.017193
Name: cinethh, dtype: float64

In [15]:
# ahora en cihispeed
hogares_del_estado['cihispeed'].value_counts(normalize=True)

yes (cable modem, fiber optic or dsl service)    0.802953
no                                               0.137006
n/a (gq)                                         0.060041
Name: cihispeed, dtype: float64

***

Esto sería el final de nuestro análisis si no estuvieramos trabajando con datos **ponderados**. Datos **ponderados** significa que cada una de nuestras observaciones representa más de una persona u hogar.

`perwt` = "Peso de persona"

`hhwt` = "Peso del hogar"

`.value_counts(normalize=True)` cuenta el número de observaciones de cada valor de tu Serie y lo divide por el total. Si cada una de nuestras observaciones fuera una persona/hogar, esta sería la respuesta a nuestra pregunta de investigación 1.

Lo que necesitamos hacer es **agregar** datos.

***

#### Paso 4: Agrupando y agregando datos

La mecánica es más o menos la misma:
1. Cuenta el número total de observaciones de cada uno de los valores de la Serie.
2. Sumas **no el número de observaciones** sino el _peso_ de cada observación.
3. Divide por el total.

#### Paso 4.1: Agrupa tus datos por sus valores correspondientes

In [16]:
hogares_del_estado.groupby("cihispeed")

De la [documentación](http://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html) de `Pandas`:

> Una operación grupal (`groupby`) implica una combinación de dividir el
objeto, **aplicar una función**, y combinar los resultados. Esto se puede usar 
para agrupar grandes cantidades de datos y operaciones de cómputo en estos grupos.

Lo que nos falta es **aplicar una función**.

Prueba lo siguiente:
```python
hogares_del_estado.groupby("countyfip").sum()
```

Podemos aplicar *casi* cualquier función.

Prueba `.mean()`, `.max()`, `.min()`, `.std()`.

In [17]:
hogares_del_estado.groupby("countyfip").sum()

,serial,hhwt,pernum,perwt
countyfip,,,,
0,4.650504e+09,446164.0,4755.0,446156.0
3,1.207007e+08,10555.0,123.0,10557.0
7,9.668951e+07,9868.0,99.0,9871.0
17,3.051483e+08,37891.0,312.0,37893.0
23,1.464338e+08,13809.0,150.0,13800.0
29,8.409909e+07,9099.0,86.0,9106.0
35,1.122669e+09,117782.0,1148.0,117759.0
41,2.369908e+08,23567.0,242.0,23568.0
45,1.693069e+08,17631.0,173.0,17622.0


También puedes seleccionar columnas como lo harías en cualquier otro DataFrame.

In [18]:
hogares_del_estado.groupby("countyfip")['hhwt'].sum()

countyfip
0      446164.0
3       10555.0
7        9868.0
17      37891.0
23      13809.0
29       9099.0
35     117782.0
41      23567.0
45      17631.0
49     132324.0
57      15170.0
61      77647.0
89      18966.0
93      31271.0
103     18136.0
109      9352.0
113     53118.0
133     15673.0
139     11236.0
153     48930.0
165     27706.0
169     10623.0
Name: hhwt, dtype: float64

***

In [19]:
n_hogares = hogares_del_estado.groupby("cihispeed")['hhwt'].sum()[2]
_estado = hogares_del_estado['statefip'].unique()[0]
print(f"""
Podemos ver ahora que {n_hogares:,} los hogares en {_estado} tienen acceso a Internet de alta velocidad. Pero, ¿de cuántos?

Para hacer esto más fácil de seguir, guardemos nuestros resultados en una variable:
""")


Podemos ver ahora que 167,114.0 los hogares en ohio tienen acceso a Internet de alta velocidad. Pero, ¿de cuántos?

Para hacer esto más fácil de seguir, guardemos nuestros resultados en una variable:



In [20]:
hogares_con_acceso_internet_alta_velocidad = hogares_del_estado.groupby("cihispeed")["hhwt"].sum()

hogares_con_acceso_internet_alta_velocidad

cihispeed
n/a (gq)                                          79136.0
yes (cable modem, fiber optic or dsl service)    910268.0
no                                               167114.0
Name: hhwt, dtype: float64

Esto se parece a cualquier 'pandas.Series' regular, ¿cómo encontramos la _suma_ total de una serie de elementos?

![math](../../static/math.png)

<details>
    <summary>respuesta</summary>
    <pre>.sum()</pre>
</details>

In [21]:
hogares_con_acceso_internet_alta_velocidad.sum()

1156518.0

¡Ese es nuestro denominador! 

![nice](../../static/nooice.gif)

***

Cuando _aplicas_ una operación a una Serie de `pandas` esta se le aplica a cada uno de sus elementos.

Prueba lo siguiente:
```python
hogares_con_acceso_internet_alta_velocidad * 1_000_000
```

```python
hogares_con_acceso_internet_alta_velocidad + 1_000_000
```

```python
hogares_con_acceso_internet_alta_velocidad / 1_000_000
```

<details>
    <summary>tip</summary>
    recuerda que puedes usar la tecla <strong>tab</strong> para autocompletar tus variables. Así no tienes que escribir todas las letras hogares_con_acceso_internet_alta_velocidad cada que lo necesites.
</details>

In [22]:
hogares_con_acceso_internet_alta_velocidad * 1_000_000

cihispeed
n/a (gq)                                         7.913600e+10
yes (cable modem, fiber optic or dsl service)    9.102680e+11
no                                               1.671140e+11
Name: hhwt, dtype: float64

In [23]:
hogares_con_acceso_internet_alta_velocidad + 1_000_000

cihispeed
n/a (gq)                                         1079136.0
yes (cable modem, fiber optic or dsl service)    1910268.0
no                                               1167114.0
Name: hhwt, dtype: float64

In [24]:
hogares_con_acceso_internet_alta_velocidad / 1_000_000

cihispeed
n/a (gq)                                         0.079136
yes (cable modem, fiber optic or dsl service)    0.910268
no                                               0.167114
Name: hhwt, dtype: float64

Ahora que tienes el denominador de nuestra ecuación (¿cuántos hogares hay en total en el estado X), cómo encontraría cada uno de los 3 valores en su participación `hogares_con_acceso_internet_alta_velocidad` del total?

In [25]:
hogares_con_acceso_internet_alta_velocidad / hogares_con_acceso_internet_alta_velocidad.sum()

cihispeed
n/a (gq)                                         0.068426
yes (cable modem, fiber optic or dsl service)    0.787076
no                                               0.144498
Name: hhwt, dtype: float64

***
***

### Parte 2 del análisis: Creando variables derivadas

Ahora que tienes la respuesta a la **Pregunta de Investigación 1**, podemos movernos a PI2:
>_¿Varía este número según los grupos demográficos? (en este caso raza/etnia)._

La función `.groupby()` de `pandas` puede tomar una lista de columnas por las cuales agrupar tu DataFrame.

Prueba lo siguiente:
```python
hogares_del_estado.groupby(['race', 'cihispeed'])[['hhwt']].sum()
```

_Notese que estoy utilizando_ `[[hhwt]]` _(una lista de un elemento) y no solo_ `[hhwt]` _prueba ambos tu y discutamos cual es la diferencia._

In [26]:
hogares_del_estado.groupby(['race', 'cihispeed'])[['hhwt']].sum()

hhwt
race                             cihispeed                                              
white                            n/a (gq)                                        57742.0
                                 yes (cable modem, fiber optic or dsl service)  742182.0
                                 no                                             124811.0
black/african american/negro     n/a (gq)                                        15443.0
                                 yes (cable modem, fiber optic or dsl service)  114753.0
                                 no                                              31265.0
american indian or alaska native n/a (gq)                                           60.0
                                 yes (cable modem, fiber optic or dsl service)    1842.0
                                 no                                                529.0
chinese                          n/a (gq)                                          117.0
                                 yes (cable modem, fiber optic or dsl service)    3233.0
                                 no                                                553.0
japanese                         n/a (gq)                                            NaN
                                 yes (cable modem, fiber optic or dsl service)    1069.0
                                 no                                                  NaN
other asian or pacific islander  n/a (gq)                                         1055.0
                                 yes (cable modem, fiber optic or dsl service)   20233.0
                                 no                                               1619.0
other race, nec                  n/a (gq)                                         2064.0
                                 yes (cable modem, fiber optic or dsl service)    8790.0
                                 no                                               4432.0
two major races                  n/a (gq)                                         2576.0
                                 yes (cable modem, fiber optic or dsl service)   15299.0
                                 no                                               3331.0
three or more major races        n/a (gq)                                           79.0
                                 yes (cable modem, fiber optic or dsl service)    2867.0
                                 no                                                574.0

In [27]:
hogares_del_estado.groupby(['race', 'cihispeed'])['hhwt'].sum()

race                              cihispeed                                    
white                             n/a (gq)                                          57742.0
                                  yes (cable modem, fiber optic or dsl service)    742182.0
                                  no                                               124811.0
black/african american/negro      n/a (gq)                                          15443.0
                                  yes (cable modem, fiber optic or dsl service)    114753.0
                                  no                                                31265.0
american indian or alaska native  n/a (gq)                                             60.0
                                  yes (cable modem, fiber optic or dsl service)      1842.0
                                  no                                                  529.0
chinese                           n/a (gq)                                            117.0


<details>
    <summary>La diferencia</summary>
    Cuando pasas una lista de elementos (aunque sea de un solo elemento - <span style='font-family:monospace;background-color:#EEE'>[[hhwt]]</span>) a un objeto <span style='font-family:monospace;background-color:#EEE'>.groupby()</span> recibes de regreso un DataFrame. Si pasas una sola columna <span style='font-family:monospace;background-color:#EEE'>[hhwt]</span>, recibes una Serie.
</details>

***

#### Paso 1: Define tus grupos

El indexador `.loc` de Pandas no solo sirve para dividir DataFrames, sino también para asignar nuevos valores a ciertos segmentos de DataFrames.

Por ejemplo,
```python
mascara_datos_inventados = (data['columna_1'] == 'Sin Respuesta')
datos.loc[mascara_datos_inventados, 'nueva_columna'] = 'esta fila no existia'
```

El código de arriba toma todas las filas que satisfacen la condición y luego busca `'nueva_columna'`, si no existe, lo creará y le asignará el valor `'esta fila no existia'` a todas las filas que satisfagan con la condición. El resto se rellenará con valores nulos (NaNs).

##### Creemos nuestras máscaras

In [28]:
mascara_latino = (hogares_del_estado['hispan'] != 'not hispanic')

In [29]:
mascara_blanco = (hogares_del_estado['hispan'] == 'not hispanic') & (hogares_del_estado['race'] == 'white')

In [30]:
mascara_negro = (hogares_del_estado['hispan'] == 'not hispanic') & (hogares_del_estado['race'].str.contains('black'))

In [31]:
mascara_nativo = (hogares_del_estado['hispan'] == 'not hispanic') & (hogares_del_estado['race'] == 'american indian or alaska native')

In [32]:
mascara_AIP = (hogares_del_estado['hispan'] == 'not hispanic') & ((hogares_del_estado['race'] >= 'chinese') & (hogares_del_estado['race'] <= 'other asian or pacific islander'))

In [33]:
mascara_otro = (hogares_del_estado['hispan'] == 'not hispanic') & (hogares_del_estado['race'] >= 'other race, nec')

<details>
    <summary>Los 6 valores</summary>
    En los círculos de investigación de Estados Unidos es común tener 5 grupos principales de raza/etnia: Latinos, Negros/Africano-Americanos, Blancos, Asiaticos/Isleños del Pacífico y Indio-Americanos/Nativos. <br>
    Esto obviamente es una sobre-simplificación. Tu puedes utilizar los grupos que quieras pero para este análisis vamos a utilizar estos 5 grupos y un grupo "Otros" para capturar todos los hogares en nuestro conjunto de datos.
</details>

Asignemos estos valores a una nueva columna `'razet'` por Raza/Etnia

In [34]:
hogares_del_estado.loc[mascara_latino, 'razet'] = 'Latino'
hogares_del_estado.loc[mascara_blanco, 'razet'] = 'Blanco'
hogares_del_estado.loc[mascara_negro, 'razet'] = 'Negro/Africano-Americano'
hogares_del_estado.loc[mascara_nativo, 'razet'] = 'Indio-Americano/Nativo'
hogares_del_estado.loc[mascara_AIP, 'razet'] = 'Asiatico/Isleños del Pacífico'
hogares_del_estado.loc[mascara_otro, 'razet'] = 'Otro'

C:\Users\sanchez\AppData\Local\Continuum\anaconda3\envs\pycon2019\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\sanchez\AppData\Local\Continuum\anaconda3\envs\pycon2019\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Revisa tus resultados.

Bajo tu nueva lógica, todos los valores de `race` deberían haber sido capturados en `razet` y no debería haber valores nulos, ¿verdad?

La función `.isna()` de Pandas regresa una Serie que contiene o True o False (verdadero / falso) correspondiendo a cada valor de la Serie dependiendo de si es un valor nulo o no.

Y además en `python`, True es igual a 1 y False a 0.

¿Qué crees que pasaría si `.sum`as los valores de una Serie de pandas booleana?

In [35]:
hogares_del_estado['razet'].isna().sum()

0

***

##### Multiples maneras de agrupar/agregar datos

Ahora que ha derivado una variable de trabajo por raza/etnia, puedes agregar tus datos para responder **PI2**. En pandas, hay muchas maneras de hacer esto, algunas de ellas son:
1. `.groupby()` como lo hemos hecho hasta ahora.
2. `.pivot_table()`
3. `pd.crosstabs()` <- este es un método `pandas`, no un método DataFrame. Más información más tarde.

##### GroupBy

In [36]:
hogares_del_estado.groupby(['razet', 'cihispeed'])[['hhwt']].sum()

hhwt
razet                         cihispeed                                              
Asiatico/Isleños del Pacífico n/a (gq)                                         1172.0
                              yes (cable modem, fiber optic or dsl service)   24535.0
                              no                                               2172.0
Blanco                        n/a (gq)                                        55415.0
                              yes (cable modem, fiber optic or dsl service)  717266.0
                              no                                             118725.0
Indio-Americano/Nativo        n/a (gq)                                           60.0
                              yes (cable modem, fiber optic or dsl service)    1140.0
                              no                                                529.0
Latino                        n/a (gq)                                         4408.0
                              yes (cable modem, fiber optic or dsl service)   37101.0
                              no                                              10413.0
Negro/Africano-Americano      n/a (gq)                                        15254.0
                              yes (cable modem, fiber optic or dsl service)  114289.0
                              no                                              30805.0
Otro                          n/a (gq)                                         2827.0
                              yes (cable modem, fiber optic or dsl service)   15937.0
                              no                                               4470.0

Guardemos eso en una variable con nombre apropiado ya que la usaremos más adelante.

In [37]:
cihispeed_por_razet = hogares_del_estado.groupby(['razet', 'cihispeed'])[['hhwt']].sum()

Ahora, este DataFrame agrupado tiene el número total de hogares en cada uno de estos grupos razet-cihispeed.

Necesitamos la proporción de los valores de `cihispeed` por grupo de razet.

En nuestra ecuación,

$$ \frac{hogares\ con\ internet\ de\ alta\ velocidad}{total\ de\ hogares\ en\ grupo\ razet}$$

Necesitamos encontrar nuestro denominador.

In [38]:
# encuentra el denominador
totales_por_razet = hogares_del_estado.groupby(['razet'])[['hhwt']].sum()

In [39]:
# divide los grupos razet-cihispeed por el denominador
cihispeed_por_razet / totales_por_razet

hhwt
razet                         cihispeed                                              
Asiatico/Isleños del Pacífico n/a (gq)                                       0.042039
                              yes (cable modem, fiber optic or dsl service)  0.880053
                              no                                             0.077908
Blanco                        n/a (gq)                                       0.062166
                              yes (cable modem, fiber optic or dsl service)  0.804646
                              no                                             0.133188
Indio-Americano/Nativo        n/a (gq)                                       0.034702
                              yes (cable modem, fiber optic or dsl service)  0.659341
                              no                                             0.305957
Latino                        n/a (gq)                                       0.084897
                              yes (cable modem, fiber optic or dsl service)  0.714553
                              no                                             0.200551
Negro/Africano-Americano      n/a (gq)                                       0.095131
                              yes (cable modem, fiber optic or dsl service)  0.712756
                              no                                             0.192113
Otro                          n/a (gq)                                       0.121675
                              yes (cable modem, fiber optic or dsl service)  0.685934
                              no                                             0.192390

In [40]:
# guardalo en una variable de nombre apropiado
proporcion_cihispeed_por_razet = cihispeed_por_razet / totales_por_razet

Este es un marco de datos de índice de múltiples niveles y hay algunas maneras de dividirlo. Probemos 3:
1. un segmento clásico `.loc`
2. una sección transversal (`.xs()`)
3. El método `.reset_index()`

**Clásico `.loc`**

In [41]:
proporcion_cihispeed_por_razet.loc[(slice(None), 'yes (cable modem, fiber optic or dsl service)'), :]

,,hhwt
razet,cihispeed,
Asiatico/Isleños del Pacífico,"yes (cable modem, fiber optic or dsl service)",0.880053
Blanco,"yes (cable modem, fiber optic or dsl service)",0.804646
Indio-Americano/Nativo,"yes (cable modem, fiber optic or dsl service)",0.659341
Latino,"yes (cable modem, fiber optic or dsl service)",0.714553
Negro/Africano-Americano,"yes (cable modem, fiber optic or dsl service)",0.712756
Otro,"yes (cable modem, fiber optic or dsl service)",0.685934


**Sección transversal**

In [42]:
proporcion_cihispeed_por_razet.xs?

Signature: proporcion_cihispeed_por_razet.xs(key, axis=0, level=None, drop_level=True)
Docstring:
Return cross-section from the Series/DataFrame.

This method takes a `key` argument to select data at a particular
level of a MultiIndex.

Parameters
----------
key : label or tuple of label
    Label contained in the index, or partially in a MultiIndex.
axis : {0 or 'index', 1 or 'columns'}, default 0
    Axis to retrieve cross-section on.
level : object, defaults to first n levels (n=1 or len(key))
    In case of a key partially contained in a MultiIndex, indicate
    which levels are used. Levels can be referred by label or position.
drop_level : bool, default True
    If False, returns object with same levels as self.

Returns
-------
Series or DataFrame
    Cross-section from the original Series or DataFrame
    corresponding to the selected index levels.

See Also
--------
DataFrame.loc : Access a group of rows and columns
    by label(s) or a boolean array.
DataFrame.iloc : Purely i

Solo necesitas una **key**, que es el valor que te interesa (por ejemplo "yes (cable modem, fiber optic or dsl service)" que es una respuesta "Si" a la pregunta "Tienes acceso al internet" del censo), y el nivel en el cual buscar esta **key** (en python empezamos a contar en 0).

In [43]:
proporcion_cihispeed_por_razet.xs(key = 'yes (cable modem, fiber optic or dsl service)', level = 1)

,hhwt
razet,
Asiatico/Isleños del Pacífico,0.880053
Blanco,0.804646
Indio-Americano/Nativo,0.659341
Latino,0.714553
Negro/Africano-Americano,0.712756
Otro,0.685934


**`.reset_index()`**

Otra forma de dividir un DataFrame de índice multinivel es convertirlo en un DataFrame de índice no multinivel. Para hacer eso necesitas _resetear_ tu índice. Después de eso, podemos dividirlo en la forma en que hemos estado segmentando nuestros DataFrames anteriormente.

In [44]:
proporcion_cihispeed_por_razet = proporcion_cihispeed_por_razet.reset_index()

In [45]:
proporcion_cihispeed_por_razet

,razet,cihispeed,hhwt
0,Asiatico/Isleños del Pacífico,n/a (gq),0.042039
1,Asiatico/Isleños del Pacífico,"yes (cable modem, fiber optic or dsl service)",0.880053
2,Asiatico/Isleños del Pacífico,no,0.077908
3,Blanco,n/a (gq),0.062166
4,Blanco,"yes (cable modem, fiber optic or dsl service)",0.804646
5,Blanco,no,0.133188
6,Indio-Americano/Nativo,n/a (gq),0.034702
7,Indio-Americano/Nativo,"yes (cable modem, fiber optic or dsl service)",0.659341
8,Indio-Americano/Nativo,no,0.305957
9,Latino,n/a (gq),0.084897


In [46]:
mascara_si_cihispeed = (proporcion_cihispeed_por_razet['cihispeed'] == 'yes (cable modem, fiber optic or dsl service)')
proporcion_cihispeed_por_razet[mascara_si_cihispeed]

,razet,cihispeed,hhwt
1,Asiatico/Isleños del Pacífico,"yes (cable modem, fiber optic or dsl service)",0.880053
4,Blanco,"yes (cable modem, fiber optic or dsl service)",0.804646
7,Indio-Americano/Nativo,"yes (cable modem, fiber optic or dsl service)",0.659341
10,Latino,"yes (cable modem, fiber optic or dsl service)",0.714553
13,Negro/Africano-Americano,"yes (cable modem, fiber optic or dsl service)",0.712756
16,Otro,"yes (cable modem, fiber optic or dsl service)",0.685934


***

##### Tablas dinámicas (pivot tables)

El segundo método para agregar nuestros datos es `.pivot_table()`s (tablas dinámicas).

Si ha trabajado con Excel, es posible que ya esté familiarizado con lo que es una tabla dinámica.

De [Wikipedia](https://es.wikipedia.org/wiki/Tabla_din%C3%A1mica):
>las tablas dinámicas pueden de forma automática clasificar, contar, totalizar o dar la media de los datos almacenados en una tabla o una hoja de cálculo. Se muestran los resultados en una segunda tabla (llamada "tabla dinámica" o "tabla pivote") que muestra los datos resumidos.

In [47]:
hogares_del_estado.pivot_table?

Signature:
hogares_del_estado.pivot_table(
    values=None,
    index=None,
    columns=None,
    aggfunc='mean',
    fill_value=None,
    margins=False,
    dropna=True,
    margins_name='All',
)
Docstring:
Create a spreadsheet-style pivot table as a DataFrame. The levels in
the pivot table will be stored in MultiIndex objects (hierarchical
indexes) on the index and columns of the result DataFrame.

Parameters
----------
values : column to aggregate, optional
index : column, Grouper, array, or list of the previous
    If an array is passed, it must be the same length as the data. The
    list can contain any of the other types (except list).
    Keys to group by on the pivot table index.  If an array is passed,
    it is being used as the same manner as column values.
columns : column, Grouper, array, or list of the previous
    If an array is passed, it must be the same length as the data. The
    list can contain any of the other types (except list).
    Keys to group by on the pivo

What we need are four things:
1. What variable will become our `index`?
2. What variable will become our `columns`?
3. What variable will become our `values`?
4. How will we aggregate our values?

Pandas is going to grab each unique value in the variables you choose and use those as rows in your `.index` or separate columns in your `.columns`. The `values` variable should be _quantitative_ in this case (but it doesn't have to be, necessarily). `.pivot_table` will by default find the `mean` of your `values` variable for each cell in your new table, in this case we don't care about the `mean`, we want to `sum` up the total number of households.

Prueba lo siguiente:

```python
hogares_del_estado.pivot_table(
    index = '______',
    columns = '______', 
    values = 'hhwt',
    aggfunc = '___',
    margins = True,
)
```

In [48]:
hogares_del_estado.pivot_table(
    index = 'razet',
    columns = 'cihispeed', 
    values = 'hhwt',
    aggfunc = 'sum',
    margins = True,
)

cihispeed,n/a (gq),"yes (cable modem, fiber optic or dsl service)",no,All
razet,,,,
Asiatico/Isleños del Pacífico,1172.0,24535.0,2172.0,27879.0
Blanco,55415.0,717266.0,118725.0,891406.0
Indio-Americano/Nativo,60.0,1140.0,529.0,1729.0
Latino,4408.0,37101.0,10413.0,51922.0
Negro/Africano-Americano,15254.0,114289.0,30805.0,160348.0
Otro,2827.0,15937.0,4470.0,23234.0
All,167114.0,79136.0,910268.0,1156518.0


Guárdalo en una variable apropiadamente nombrada.

In [49]:
hogares_tabla_pivote = hogares_del_estado.pivot_table(
    index = 'razet',
    columns = 'cihispeed', 
    values = 'hhwt',
    aggfunc = 'sum',
    margins = True,
)

¿Cúal crees que es el siguiente paso?

In [50]:
hogares_tabla_pivote['yes (cable modem, fiber optic or dsl service)'] / hogares_tabla_pivote['All']

razet
Asiatico/Isleños del Pacífico    0.880053
Blanco                           0.804646
Indio-Americano/Nativo           0.659341
Latino                           0.714553
Negro/Africano-Americano         0.712756
Otro                             0.685934
All                              0.068426
dtype: float64